# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import nltk
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from sqlalchemy import create_engine
import sqlite3
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier


In [2]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')

connection = engine.connect()
query = "SELECT * FROM DisasterMessages"
df = pd.read_sql(query, connection)
connection.close()

df.shape

(26216, 39)

In [3]:
# Select X for analysis

X = df["message"].values

In [4]:
# Select Y for analysis

N_CATEGORIES = 36 #number of category columns
df_y = df.iloc[:,-N_CATEGORIES:]
df_y_sufficient_data =  df_y.loc[:,(df_y.sum(axis=0) > 50)]
Y = df_y_sufficient_data.values
N_CATEGORIES = df_y_sufficient_data.shape[1]
N_CATEGORIES # Columns with sufficient values
# df_y_sufficient_data.columns

35

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # Bring to lowercase, remove stopwords and punctuation
    stop = set(stopwords.words('english') + list(string.punctuation))
    nltk_tokens = nltk.word_tokenize(text.lower())
    tokens = [w for w in nltk_tokens if not w in stop]
    return(tokens)


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize)
mo = MultiOutputClassifier(LogisticRegression(solver="liblinear"))
pipeline = Pipeline([('tfidf', tfidf_vectorizer), ('mo', mo)])

### 4. Train pipeline
- Split data into train and test sets


In [7]:
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=0.2, random_state=42)

- Train pipeline

In [8]:
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x000001693A835DC0>)),
                ('mo',
                 MultiOutputClassifier(estimator=LogisticRegression(solver='liblinear')))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
Y_pred = pipeline.predict(X_validation)

Report the classification results

In [11]:
classification_result = list()

for i in range(0,N_CATEGORIES):
    # classification_result.append(classification_report(Y_validation[:,i], Y_pred[:,i],zero_division=1))
    weighted_avg = classification_report(Y_validation[:,i], Y_pred[:,i],zero_division=1,output_dict=True)['weighted avg']
    classification_result.append(weighted_avg)

df_result = pd.DataFrame(classification_result)

,precision,recall,f1-score,support
0,0.803729,0.811976,0.787331,5244
1,0.896821,0.901602,0.892424,5244
2,0.995067,0.995042,0.992569,5244
3,0.783312,0.784134,0.780704,5244
4,0.910415,0.926201,0.904742,5244
5,0.949076,0.954424,0.938893,5244
6,0.977459,0.976926,0.966565,5244
7,0.983501,0.983219,0.974899,5244
8,0.959170,0.970633,0.959644,5244
9,0.952829,0.957475,0.952663,5244


Show the distribution of the results

In [13]:
df_result.describe()

,precision,recall,f1-score,support
count,35.000000,35.000000,35.000000,35.0
mean,0.941101,0.947156,0.935978,5244.0
std,0.053897,0.051308,0.055210,0.0
min,0.783312,0.784134,0.780704,5244.0
25%,0.924261,0.938406,0.923418,5244.0
50%,0.952829,0.957475,0.952663,5244.0
75%,0.980101,0.981789,0.973702,5244.0
max,0.995822,0.995805,0.993712,5244.0


### 6. Improve your model
Use grid search to find better parameters. 

In [29]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize)
mo = MultiOutputClassifier(LogisticRegression(solver="liblinear"))
pipeline = Pipeline([('tfidf', tfidf_vectorizer), ('mo', mo)])

parameters = {
    'tfidf__sublinear_tf': [True,False],
}

cv = model_selection.GridSearchCV(pipeline, parameters)

cv.fit(X_train, Y_train)

GridSearchCV(estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x000001693A835DC0>)),
                                       ('mo',
                                        MultiOutputClassifier(estimator=LogisticRegression(solver='liblinear')))]),
             param_grid={'tfidf__sublinear_tf': [True, False]})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [30]:
Y_pred =cv.predict(X_validation)

classification_result_cv = list()

for i in range(0,N_CATEGORIES):
    # classification_result.append(classification_report(Y_validation[:,i], Y_pred[:,i],zero_division=1))
    weighted_avg = classification_report(Y_validation[:,i], Y_pred[:,i],zero_division=1,output_dict=True)['weighted avg']
    classification_result_cv.append(weighted_avg)

df_result_cv = pd.DataFrame(classification_result_cv)
df_result_cv.describe()

,precision,recall,f1-score,support
count,35.000000,35.000000,35.000000,35.0
mean,0.941086,0.947172,0.935982,5244.0
std,0.053996,0.051300,0.055100,0.0
min,0.782408,0.783371,0.780067,5244.0
25%,0.922757,0.938692,0.923119,5244.0
50%,0.952343,0.957285,0.952423,5244.0
75%,0.980480,0.981884,0.973759,5244.0
max,0.995822,0.995805,0.993712,5244.0


In [31]:
best_parameters = cv.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

	tfidf__sublinear_tf: True


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.